###  3. House prices model

In this exercise, you'll interpret your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

* Load the **houseprices** data from Thinkful's database.
* Run your house prices model again and interpret the results. Which features are statistically significant, and which are not?
* Now, exclude the insignificant features from your model. Did anything change?
* Interpret the statistically significant coefficients by quantifying their relations with the house prices. Which features have a more prominent effect on house prices?
* Do the results sound reasonable to you? If not, try to explain the potential reasons.

In [3]:
# %load 19.2_linreg_drill.py
#!/usr/bin/env python
'''
output is lrm model
'''
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from scipy.stats import mode

#credentials
user = 'dsbc_student'
pw = '7*.8G9QH21'
host = '142.93.121.174'
port = '5432'
db = 'houseprices'
dialect = 'postgresql'

engine = create_engine('{}://{}:{}@{}:{}/{}'.format(dialect, user, pw, host, port, db))
engine.table_names()

sql_query = '''
SELECT
    *
FROM
    houseprices
'''
source_df = pd.read_sql(sql_query, con=engine)
engine.dispose()
house_df = source_df.copy()
for column in house_df.columns[house_df.dtypes== 'object']:
    print("Column {} has values {}".format(column, house_df[column].unique()))
#fillvalues
missing_numerical = ['lotfrontage', 'masvnrarea', 'garageyrblt']
for miss in missing_numerical: #column-wise
    house_df[miss] = house_df[miss].fillna(house_df[miss].mean()) #fill with column mean
    
missing_cat_ob = house_df.dtypes[house_df.isna().sum() > 0]
missing_categorical = missing_cat_ob[missing_cat_ob == 'object'].index
for miss in missing_categorical:
    house_df[miss] = house_df[miss].fillna(house_df[miss].value_counts().index[0])  #fill with most common value

categorical_feat = house_df.dtypes[house_df.dtypes == 'object'].index
new_categories_df = pd.DataFrame()
for feature in categorical_feat:
    new_categories_df = pd.concat([new_categories_df, 
                                   pd.get_dummies(house_df[feature], columns=categorical_feat, drop_first=True)], axis=1)
new_categories_df = pd.concat([new_categories_df, 
                               house_df.filter(items=(house_df.columns[(house_df.dtypes.values != 'object').tolist()]), axis=1) ], 
                              axis=1) #tolist() needed to avoid hashability issue

#Find highly (>.95) correlated values and drop
house_corr_df = house_df.corr()
house_corr_df[house_corr_df >.95].notna()#.any()

#standardize data and compute PCA
pca = PCA()
scaler = StandardScaler()
X = scaler.fit_transform(new_categories_df.drop(["saleprice"], axis=1))
y = new_categories_df.saleprice

pca.fit(X)
pca.explained_variance_ratio_

sns.set_style('darkgrid')
plt.figure(figsize=(15,5))
sns.lineplot(data=np.cumsum(pca.explained_variance_ratio_), marker="o")
plt.title("Cumulative Variance explained");

pca_75 = PCA(n_components=75)
X_pca = pca_75.fit_transform(X)
lrm = linear_model.LinearRegression()
lrm.fit(X_pca, y)
lrm

print('\nCoefficients: \n', lrm.coef_)
print('\nIntercept: \n', lrm.intercept_)

Column mszoning has values ['RL' 'RM' 'C (all)' 'FV' 'RH']
Column street has values ['Pave' 'Grvl']
Column alley has values [None 'Grvl' 'Pave']
Column lotshape has values ['Reg' 'IR1' 'IR2' 'IR3']
Column landcontour has values ['Lvl' 'Bnk' 'Low' 'HLS']
Column utilities has values ['AllPub' 'NoSeWa']
Column lotconfig has values ['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']
Column landslope has values ['Gtl' 'Mod' 'Sev']
Column neighborhood has values ['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']
Column condition1 has values ['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe']
Column condition2 has values ['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe']
Column bldgtype has values ['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']
Column housestyle has values ['2Story' '1Stor

In [4]:
%whos

Variable              Type                Data/Info
---------------------------------------------------
PCA                   ABCMeta             <class 'sklearn.decomposition.pca.PCA'>
StandardScaler        type                <class 'sklearn.preproces<...>ing.data.StandardScaler'>
X                     ndarray             1460x246: 359160 elems, type `float64`, 2873280 bytes (2.74017333984375 Mb)
X_pca                 ndarray             1460x75: 109500 elems, type `float64`, 876000 bytes (855.46875 kb)
categorical_feat      Index               Index(['mszoning', 'stree<...>],\n      dtype='object')
column                str                 salecondition
create_engine         function            <function create_engine at 0x00000178791E8E58>
db                    str                 houseprices
dialect               str                 postgresql
engine                Engine              Engine(postgresql://dsbc_<...>121.174:5432/houseprices)
feature               str                

In [5]:
lrm.coef_

array([15293.71342719,  3213.25273159, 10710.83003588,  4570.78886077,
         104.42211267,  5796.47721072, -5892.67746355,  1314.22816376,
       -1684.95680329, -2825.37836074,  -996.53745537,  1631.22594528,
        -477.15540483,   602.32879365,   400.64971486,   926.27880386,
        -624.26414729, -1373.72880582,  1834.13458121,   383.13225243,
         251.91665618, -1545.10993489,   839.00791669,  2600.21643291,
       -1427.23758014, -1446.22917025, -1750.64877197,  2481.44463395,
        3480.91479322,  1381.21978791,  -890.00871042,   -86.21154098,
          48.94140008, -1542.47398565,  4281.83118693,   330.21770707,
        -758.92889523,  -877.08418645, -1777.19083407,  1095.2728767 ,
         886.94973483,  -720.6872267 , -1794.74150167,  2168.70980048,
       -2130.08825175,  2972.64676821,  1951.25821544, -1549.84011548,
        1042.5869979 ,  2708.23185119,  1102.5274296 , -3402.06368852,
       -1553.50021346,  1695.91544799,  1171.9439866 , -1243.61672007,
      

Since I added all the variables in and did a PCA and selected 75 variables, i'm not sure they have an obvious interpretation. 

In [8]:
import statsmodels.api as sm
sm.add_constant(X_pca)
results = sm.OLS(y, X_pca).fit()

In [26]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              saleprice   R-squared (uncentered):                   0.135
Model:                            OLS   Adj. R-squared (uncentered):              0.088
Method:                 Least Squares   F-statistic:                              2.888
Date:                Tue, 23 Jul 2019   Prob (F-statistic):                    3.21e-14
Time:                        11:19:25   Log-Likelihood:                         -19769.
No. Observations:                1460   AIC:                                  3.969e+04
Df Residuals:                    1385   BIC:                                  4.008e+04
Df Model:                          75                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.529e+04   1227.669     12.458      0.000    1.29e+04    1.77e+04
x2          3213.2527   1795.544      1.790      0.074    -309.026    6735.532
x3          1.071e+04   2026.164      5.286      0.000    6736.148    1.47e+04
x4          4570.7889   2179.485      2.097      0.036     295.341    8846.237
x5           104.4221   2436.355      0.043      0.966   -4674.923    4883.767
x6          5796.4772   2502.834      2.316      0.021     886.722    1.07e+04
x7         -5892.6775   2592.880     -2.273      0.023    -1.1e+04    -806.281
x8          1314.2282   2695.137      0.488      0.626   -3972.764    6601.220
x9         -1684.9568   2701.461     -0.624      0.533   -6984.353    3614.440
x10        -2825.3784   2834.273     -0.997      0.319   -8385.310    2734.553
x11         -996.5375   2854.585     -0.349      0.727   -6596.316    4603.241
x12         1631.2259   2910.627      0.560      0.575   -4078.488    7340.940
x13         -477.1554   2967.637     -0.161      0.872   -6298.704    5344.394
x14          602.3288   2977.143      0.202      0.840   -5237.867    6442.525
x15          400.6497   3031.412      0.132      0.895   -5546.005    6347.304
x16          926.2788   3094.967      0.299      0.765   -5145.050    6997.608
x17         -624.2641   3104.964     -0.201      0.841   -6715.205    5466.677
x18        -1373.7288   3145.758     -0.437      0.662   -7544.695    4797.237
x19         1834.1346   3158.435      0.581      0.562   -4361.699    8029.968
x20          383.1323   3238.357      0.118      0.906   -5969.482    6735.747
x21          251.9167   3275.989      0.077      0.939   -6174.520    6678.353
x22        -1545.1099   3314.910     -0.466      0.641   -8047.896    4957.676
x23          839.0079   3326.704      0.252      0.801   -5686.915    7364.931
x24         2600.2164   3374.701      0.771      0.441   -4019.861    9220.294
x25        -1427.2376   3382.421     -0.422      0.673   -8062.460    5207.984
x26        -1446.2292   3440.412     -0.420      0.674   -8195.211    5302.752
x27        -1750.6488   3485.609     -0.502      0.616   -8588.292    5086.995
x28         2481.4446   3488.598      0.711      0.477   -4362.062    9324.951
x29         3480.9148   3509.243      0.992      0.321   -3403.092    1.04e+04
x30         1381.2198   3574.658      0.386      0.699   -5631.109    8393.548
x31         -890.0087   3590.824     -0.248      0.804   -7934.050    6154.033
x32          -86.2115   3608.118     -0.024      0.981   -7164.178    6991.755
x33           48.9414   3633.942      0.013      0.989   -7079.684    7177.566
x34        -1542.4740   3681.763     -0.419      0.675   -8764.909    5679.961
x35         4281.8312   3714.801      1.153      0.249   -300

In [23]:
results.pvalues[results.pvalues <.1 ]

x1    7.683273e-34
x2    7.374132e-02
x3    1.448916e-07
x4    3.615789e-02
x6    2.070557e-02
x7    2.320069e-02
dtype: float64

In [24]:
pca_limited = PCA(n_components=4) #QUESTION: how to do a PCA skipping a component???A: just don't. 
X_pca_limited = pca_limited.fit_transform(X)
sm.add_constant(X_pca_limited)
results_limited = sm.OLS(y, X_pca_limited).fit()

In [25]:
results_limited.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              saleprice   R-squared (uncentered):                   0.119
Model:                            OLS   Adj. R-squared (uncentered):              0.117
Method:                 Least Squares   F-statistic:                              49.22
Date:                Tue, 23 Jul 2019   Prob (F-statistic):                    7.00e-39
Time:                        11:19:01   Log-Likelihood:                         -19782.
No. Observations:                1460   AIC:                                  3.957e+04
Df Residuals:                    1456   BIC:                                  3.959e+04
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.529e+04   1208.470     12.655      0.000    1.29e+04    1.77e+04
x2          3213.9114   1767.464      1.818      0.069    -253.136    6680.958
x3          1.071e+04   1994.479      5.370      0.000    6798.594    1.46e+04
x4          4587.5557   2145.428      2.138      0.033     379.095    8796.016
==============================================================================
Omnibus:                      506.277   Durbin-Watson:                   0.094
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            26541.479
Skew:                           0.816   Prob(JB):                         0.00
Kurtosis:                      23.824   Cond. No.                         1.78
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The coefficients changed, but are remarkably similar to the original ones!